# ARCO/RCI Demo for XRD Patterns

This notebook demonstrates the use of ARCO (Arcogram with Rational Coherence) and RCI (Rational Coherence Index) for analyzing X-ray diffraction patterns.

## Overview

ARCO converts 1D signals (XRD intensity profiles) into fixed-length interpretable fingerprints by:
1. Computing local power spectra (STFT-style sliding windows)
2. Integrating spectral power around rational-frequency anchors (Farey set)
3. Computing RCI: fraction of energy concentrated on major rational arcs

## Why This Matters for XRD

Low-denominator rationals correspond to periodic patterns in diffraction:
- Regular lattice spacing → strong rational frequencies
- Peak spacing relationships → integer period ratios
- Robustness to noise and peak jitter

In [ ]:
# Import required libraries
import numpy as np
import matplotlib.pyplot as plt
import sys
import os

# Add lib directory to path
sys.path.insert(0, os.path.abspath('..'))

from nomad_auto_xrd.common.arco_rci import ARCO, generate_anchors, get_anchor_interpretation_table

print("ARCO/RCI module loaded successfully!")

## 1. Generate Rational Anchors

For XRD, we use `Qmax=40` to capture longer-period relationships in diffraction patterns.

In [ ]:
# Generate rational anchors for XRD (higher Qmax for longer periodicities)
anchors = generate_anchors(Qmax=40)

print(f"Generated {len(anchors)} rational anchors")
print(f"First 10 anchors: {anchors[:10]}")

# Visualize anchor distribution
plt.figure(figsize=(12, 4))
plt.plot(anchors, np.ones_like(anchors), '|', markersize=20, color='blue')
plt.xlabel('Normalized Frequency (cycles/sample)')
plt.ylabel('Rational Anchors')
plt.title(f'Distribution of {len(anchors)} Rational Anchors (Qmax=40)')
plt.grid(True, alpha=0.3)
plt.ylim(0.5, 1.5)
plt.show()

## 2. Synthetic XRD Pattern: Equidistant Peaks

Create a synthetic XRD pattern with regularly spaced peaks (simulating a simple crystalline lattice).

In [ ]:
# Create synthetic XRD pattern with equidistant peaks
def create_synthetic_xrd(length=2048, peak_spacing=8, num_peaks=20, noise_level=0.1):
    """Create synthetic XRD pattern with periodic peaks."""
    pattern = np.zeros(length)
    
    # Add equidistant peaks
    start = length // 4
    for i in range(num_peaks):
        center = start + i * peak_spacing
        if center < length:
            # Gaussian peak
            x = np.arange(length)
            pattern += np.exp(-0.5 * ((x - center) / 2.0) ** 2)
    
    # Add noise
    pattern += noise_level * np.random.randn(length)
    
    # Ensure positive values
    pattern = np.maximum(pattern, 0)
    
    return pattern

# Generate patterns
xrd_periodic = create_synthetic_xrd(length=2048, peak_spacing=8, noise_level=0.05)
xrd_random = np.random.rand(2048) * 0.5 + 0.1  # Random noise pattern

# Plot patterns
fig, axes = plt.subplots(2, 1, figsize=(14, 8))

axes[0].plot(xrd_periodic, linewidth=1)
axes[0].set_title('Synthetic XRD: Periodic Peaks (Simulated Crystalline)')
axes[0].set_xlabel('Position (arbitrary units)')
axes[0].set_ylabel('Intensity')
axes[0].grid(True, alpha=0.3)

axes[1].plot(xrd_random, linewidth=1, color='orange')
axes[1].set_title('Random XRD: Noise Pattern (Simulated Amorphous)')
axes[1].set_xlabel('Position (arbitrary units)')
axes[1].set_ylabel('Intensity')
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## 3. Initialize ARCO Analyzer

Parameters for XRD:
- `Qmax=40`: Capture longer periodicities
- `window_sizes=[128, 256]`: Multi-scale analysis
- `alpha=1.0`: Bandwidth scaling factor

In [ ]:
# Initialize ARCO analyzer
arco = ARCO(
    anchors=anchors,
    window_sizes=[128, 256],
    hop_fraction=0.25,
    alpha=1.0,
    apply_hann=True
)

print("ARCO analyzer initialized")
print(f"Number of anchors: {len(anchors)}")
print(f"Window sizes: {arco.window_sizes}")
print(f"Expected ARCO-print length: {len(anchors) * len(arco.window_sizes)}")

## 4. Compute ARCO-print and RCI

In [ ]:
# Compute ARCO features for both patterns
arco_print_periodic = arco.compute_arco_print({'intensity': xrd_periodic})
rci_periodic = arco.compute_rci({'intensity': xrd_periodic}, major_q=20)

arco_print_random = arco.compute_arco_print({'intensity': xrd_random})
rci_random = arco.compute_rci({'intensity': xrd_random}, major_q=20)

print("\n=== Periodic XRD Pattern ===")
print(f"ARCO-print shape: {arco_print_periodic.shape}")
print(f"RCI: {rci_periodic:.4f}")

print("\n=== Random XRD Pattern ===")
print(f"ARCO-print shape: {arco_print_random.shape}")
print(f"RCI: {rci_random:.4f}")

print(f"\n** RCI Ratio (Periodic/Random): {rci_periodic/rci_random:.2f}x **")
print("Higher RCI indicates stronger periodicity/crystallinity")

## 5. Visualize ARCO-prints

In [ ]:
# Plot ARCO-prints
fig, axes = plt.subplots(2, 1, figsize=(14, 8))

# Periodic pattern
axes[0].bar(range(len(arco_print_periodic)), arco_print_periodic, 
           color='steelblue', alpha=0.7)
axes[0].set_title(f'ARCO-print: Periodic XRD (RCI={rci_periodic:.4f})')
axes[0].set_xlabel('Feature Index (Anchor × Window Scale)')
axes[0].set_ylabel('Arc Power')
axes[0].grid(True, alpha=0.3)

# Random pattern
axes[1].bar(range(len(arco_print_random)), arco_print_random, 
           color='orange', alpha=0.7)
axes[1].set_title(f'ARCO-print: Random XRD (RCI={rci_random:.4f})')
axes[1].set_xlabel('Feature Index (Anchor × Window Scale)')
axes[1].set_ylabel('Arc Power')
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## 6. Null Model Z-score

Validate RCI significance by comparing to shuffled null distribution.

In [ ]:
# Compute null model z-score for periodic pattern
z_score_periodic, p_value_periodic = arco.null_model_zscore(
    xrd_periodic, n_shuffles=50, preserve_composition=True
)

z_score_random, p_value_random = arco.null_model_zscore(
    xrd_random, n_shuffles=50, preserve_composition=True
)

print("\n=== Null Model Validation ===")
print(f"\nPeriodic XRD:")
print(f"  Z-score: {z_score_periodic:.2f}")
print(f"  P-value: {p_value_periodic:.4f}")
print(f"  Interpretation: {'SIGNIFICANT periodicity' if z_score_periodic > 2 else 'No significant periodicity'}")

print(f"\nRandom XRD:")
print(f"  Z-score: {z_score_random:.2f}")
print(f"  P-value: {p_value_random:.4f}")
print(f"  Interpretation: {'SIGNIFICANT periodicity' if z_score_random > 2 else 'No significant periodicity'}")

## 7. ARCO-3D: Position-Resolved Analysis

Visualize how ARCO features evolve across the pattern.

In [ ]:
# Compute ARCO-3D
arco_3d_periodic = arco.compute_arco_3d({'intensity': xrd_periodic}, finest_window=128)

print(f"ARCO-3D shape: {arco_3d_periodic.shape}")
print(f"(n_tracks={arco_3d_periodic.shape[0]}, n_windows={arco_3d_periodic.shape[1]}, n_anchors={arco_3d_periodic.shape[2]})")

# Plot heatmap
plt.figure(figsize=(14, 6))
plt.imshow(arco_3d_periodic[0].T, aspect='auto', cmap='viridis', interpolation='nearest')
plt.colorbar(label='Arc Power')
plt.xlabel('Window Position')
plt.ylabel('Rational Anchor Index')
plt.title('ARCO-3D: Position-Resolved Arcogram (Periodic XRD)')
plt.tight_layout()
plt.show()

## 8. Interpretation Guide

Common rational anchors and their typical meanings.

In [ ]:
interpretation_table = get_anchor_interpretation_table()

print("\n=== Rational Anchor Interpretation Guide ===")
print(f"{'Rational':<12} {'Period':<10} {'Typical Meaning'}")
print("=" * 80)
for anchor, info in interpretation_table.items():
    print(f"{anchor:<12.4f} {info['period']:<10} {info['typical_meaning']}")

## Conclusion

This notebook demonstrated:
1. ✅ Generating rational anchors for XRD analysis
2. ✅ Computing ARCO-print fingerprints
3. ✅ Computing RCI (Rational Coherence Index)
4. ✅ Validating significance with null models
5. ✅ Position-resolved ARCO-3D analysis

### Key Takeaways for XRD:
- **High RCI** → strong periodicity → crystalline material
- **Low RCI** → weak periodicity → amorphous material
- **ARCO-print** → fixed-length fingerprint for similarity search, ML features
- **ARCO-3D** → localization of periodic features along the pattern